<a href="https://colab.research.google.com/github/ifeLight/ml-bot/blob/main/binance-multi-timeframe-grade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas-ta
!pip install backtrader[plotting]
!pip install plotly
!pip install --upgrade firebase-admin

In [4]:
from google.colab import auth
import google.auth
auth.authenticate_user()
#Configure Google cloud project
project_id = 'ifelight'
!gcloud config set project {project_id}

Updated property [core/project].


In [5]:
import numpy as np
import pandas as pd
import datetime
import requests
import json
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import backend as K
import backtrader as bt
import pandas_ta as ta
from imblearn.over_sampling import SMOTE
import subprocess
import firebase_admin
from firebase_admin import firestore
import hashlib
import gc

In [6]:
# Working GCP Bucket name
bucket_name = 'ife-storage'

In [7]:
# Working on Firestore name
firestore_collection_name = 'trade-models'

In [8]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

binance_base_url = 'https://52on3577u3.execute-api.eu-central-1.amazonaws.com'

def get_binance_candles(symbol: str, interval='1h', limit: int = 50, **kwargs):
    url = f'{binance_base_url}/api/v3/uiKlines?symbol={symbol}&interval={interval}&limit={limit}'
    for key, value in kwargs.items():
        url += f'&{key}={value}'
    response = requests.get(url)
    result = json.loads(response.text)
    # print(result)
    def map_result(x):
        return {
            'Date': x[0],
            'Open': x[1],
            'High': x[2],
            'Low': x[3],
            'Close': x[4],
            'Volume': x[5],
        }
    mappeded_result = []
    for x in result:
        mappeded_result.append(map_result(x))
    return mappeded_result


def candles_to_df(data):
    df =  pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'], unit='ms')
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df.set_index('Date', inplace=True)
    return df

def get_all_binance_candles(symbol: str, interval='1h', start_date=None, end_date=None, limit=1000):
  raw_start_date = start_date
  raw_end_date = end_date
  start_date = pd.to_datetime(start_date) if start_date else pd.to_datetime('2015-01-01')
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')
  try:
    if end_date <= pd.to_datetime('today'):
      return load_candles_from_cloud_storage(symbol, interval, start_date, end_date)
  except FileNotFoundError:
    pass
  result = []
  while True:
    candles = get_binance_candles(symbol, interval, limit, startTime=int(start_date.timestamp() * 1000), endTime=int(end_date.timestamp() * 1000))
    if len(candles) <= 1:
      break;
    result += candles
    start_date = pd.to_datetime(datetime.datetime.fromtimestamp(candles[-1]['Date'] / 1000))
  candles_df = candles_to_df(result)
  save_candles_to_cloud_storage(candles_df, symbol, interval, raw_start_date, raw_end_date)
  return candles_df

def candles_storage_file_name(symbol: str, interval='1h', start_date=None, end_date=None):
  file_name = f'binance_{symbol}_{interval}_{start_date}_{end_date}.csv'
  return file_name

def load_candles_from_cloud_storage(symbol: str, interval: str, start_date=None, end_date=None):
  file_name = candles_storage_file_name(symbol, interval, start_date, end_date)
  try:
    # Download the file from cloud storage.
    subprocess.run(['gsutil', 'cp', f'gs://{bucket_name}/trade/candles/{file_name}', f'/tmp/{file_name}'], check=True)

    # Load the data into a Pandas DataFrame.
    with open(f'/tmp/{file_name}', 'r') as f:
      return pd.read_csv(f, index_col=0, parse_dates=True)
  except subprocess.CalledProcessError:
    # Raise a FileNotFoundError if the file is not found in cloud storage.
    raise FileNotFoundError(f"File not found: gs://{bucket_name}/trade/candles/{file_name}")

def save_candles_to_cloud_storage(df: pd.DataFrame, symbol: str, interval: str, start_date, end_date):
  file_name = candles_storage_file_name(symbol, interval, start_date, end_date)
  df.to_csv(f'/tmp/{file_name}')
  !gsutil cp /tmp/{file_name} gs://{bucket_name}/trade/candles/{file_name}


In [9]:
timeframe = '5m'
symbol = 'BTCUSDT'
exchange = 'binance'
start_date = '01-01-2024'
end_date = '28-03-2025'
raw_data = get_all_binance_candles(symbol, timeframe, start_date, end_date)
raw_data

<ipython-input-8-fa8437b8f1e0>:43: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')


Copying file:///tmp/binance_BTCUSDT_5m_01-01-2024_28-03-2025.csv [Content-Type=text/csv]...
\
Operation completed over 1 objects/7.8 MiB.                                      


,Open,High,Low,Close,Volume
Date,,,,,
2024-01-01 00:00:00,42283.58,42397.23,42261.02,42397.23,155.25731
2024-01-01 00:05:00,42397.22,42432.74,42385.26,42409.96,141.31102
2024-01-01 00:10:00,42409.97,42488.09,42409.96,42488.00,135.14249
2024-01-01 00:15:00,42488.00,42554.57,42458.58,42458.85,163.32541
2024-01-01 00:20:00,42458.85,42491.10,42445.84,42473.93,117.27112
...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156


# New Section

In [10]:
data = raw_data.copy()
data

,Open,High,Low,Close,Volume
Date,,,,,
2024-01-01 00:00:00,42283.58,42397.23,42261.02,42397.23,155.25731
2024-01-01 00:05:00,42397.22,42432.74,42385.26,42409.96,141.31102
2024-01-01 00:10:00,42409.97,42488.09,42409.96,42488.00,135.14249
2024-01-01 00:15:00,42488.00,42554.57,42458.58,42458.85,163.32541
2024-01-01 00:20:00,42458.85,42491.10,42445.84,42473.93,117.27112
...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156


In [11]:
def add_pivots(df, window=5):
    """
    Calculate the probability of price going up and down based on higher and lower pivots.
    """
    df['Higher_Pivot'] = df['High'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: 1 if x.iloc[window] == x.max() else 0, raw=False
    )
    df['Lower_Pivot'] = df['Low'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: 1 if x.iloc[window] == x.min() else 0, raw=False
    )

    # Step 2: Ensure no two successive pivots of the same type
    pivot_type = None  # Tracks the type of the last pivot
    last_pivot_index = None  # Tracks the index of the last pivot

    # Remove duplicated index
    df = df[~df.index.duplicated(keep='first')]

    for i in range(len(df.index)):
      if df.loc[df.index[i], 'Higher_Pivot'].item() == 1:
          if pivot_type == 'higher':
              # Compare current higher pivot with the last higher pivot
              if df.loc[df.index[i], 'High'].item() > df.loc[df.index[last_pivot_index], 'High'].item():
                  # Remove the last higher pivot
                  df.loc[df.index[last_pivot_index], 'Higher_Pivot'] = 0
                  # Update the last pivot
                  last_pivot_index = i
              else:
                  # Remove the current higher pivot
                  df.loc[df.index[i], 'Higher_Pivot'] = 0
          else:
              # Update pivot type and index
              pivot_type = 'higher'
              last_pivot_index = i

      elif df.loc[df.index[i], 'Lower_Pivot'].item() == 1:
          if pivot_type == 'lower':
              # Compare current lower pivot with the last lower pivot
              if df.loc[df.index[i], 'Low'].item() < df.loc[df.index[last_pivot_index], 'Low'].item():
                  # Remove the last lower pivot
                  df.loc[df.index[last_pivot_index], 'Lower_Pivot'] = 0
                  # Update the last pivot
                  last_pivot_index = i
              else:
                  # Remove the current lower pivot
                  df.loc[df.index[i], 'Lower_Pivot'] = 0
          else:
              # Update pivot type and index
              pivot_type = 'lower'
              last_pivot_index = i
    return df

In [12]:
data = add_pivots(data, 10)

In [13]:
def plot_pivots(df):
  candlestick = go.Candlestick(
      x=df.index,
      open=df['Open'],
      high=df['High'],
      low=df['Low'],
      close=df['Close'],
      name='Candlestick'
  )

  # Step 3: Add markers for higher and lower pivots
  higher_pivots_df = df[df['Higher_Pivot'] == 1]
  lower_pivots_df = df[df['Lower_Pivot'] == 1]

  higher_pivots = go.Scatter(
      x=higher_pivots_df.index,
      y=higher_pivots_df['High'],
      mode='markers',
      marker=dict(color='red', size=10, symbol='triangle-down'),
      name='Higher Pivot'
  )

  lower_pivots = go.Scatter(
      x=lower_pivots_df.index,
      y=lower_pivots_df['Low'],
      mode='markers',
      marker=dict(color='green', size=10, symbol='triangle-up'),
      name='Lower Pivot'
  )

  # Step 4: Create the figure
  fig = go.Figure(data=[candlestick, higher_pivots, lower_pivots])

  # Update layout
  fig.update_layout(
      title='Candlestick Chart with Higher and Lower Pivots',
      xaxis_title='Date',
      yaxis_title='Price',
      xaxis_rangeslider_visible=False,
      template='plotly_dark'
  )

  # Show the plot
  fig.show()

In [14]:
plot_pivots(data[:1000])

In [15]:
data['Pivot'] = np.where(data['Higher_Pivot'] == 1, -1, np.where(data['Lower_Pivot'] == 1, 1, 0))
del data['Higher_Pivot']
del data['Lower_Pivot']
data

<ipython-input-15-4b01a0a1c0f2>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Open,High,Low,Close,Volume,Pivot
Date,,,,,,
2024-01-01 00:00:00,42283.58,42397.23,42261.02,42397.23,155.25731,0
2024-01-01 00:05:00,42397.22,42432.74,42385.26,42409.96,141.31102,0
2024-01-01 00:10:00,42409.97,42488.09,42409.96,42488.00,135.14249,0
2024-01-01 00:15:00,42488.00,42554.57,42458.58,42458.85,163.32541,0
2024-01-01 00:20:00,42458.85,42491.10,42445.84,42473.93,117.27112,0
...,...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035,0
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118,0
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156,0


In [16]:
def calculate_pivot_proximity(df):
    """
    Calculates the pivot proximity using a loop-based approach, finding the closest
    previous and next pivots without generating intermediate lists of all pivots.

    Args:
        df (pd.DataFrame): DataFrame with 'Pivot' and 'Close' columns.

    Returns:
        pd.DataFrame: DataFrame with 'Pivot_Proximity' column added.
    """
    df = df.copy()

    pivot_values = df['Pivot'].values
    close_values = df['Close'].values
    proximity_values = [0.0] * len(df)

    for i in range(len(df)):
      pivot = pivot_values[i]
      if pivot == -1:
        proximity_values[i] = -1.0
      elif pivot == 1:
        proximity_values[i] = 1.0
      else:
        closest_previous_pivot_index = None
        for j in range(i - 1, -1, -1):
          if pivot_values[j] != 0:
            closest_previous_pivot_index = j
            break

        if closest_previous_pivot_index is not None:
          closest_previous_pivot_value = pivot_values[closest_previous_pivot_index]
          closest_previous_close = close_values[closest_previous_pivot_index]

          closest_next_pivot_index = None
          if closest_previous_pivot_value == -1:
            for j in range(i + 1, len(df)):
              if pivot_values[j] == 1:
                closest_next_pivot_index = j
                break
          else:
            for j in range(i + 1, len(df)):
              if pivot_values[j] == -1:
                closest_next_pivot_index = j
                break

          if closest_next_pivot_index is not None:
            closest_next_close = close_values[closest_next_pivot_index]

            distance_to_previous = abs(close_values[i] - closest_previous_close)
            distance_to_next = abs(close_values[i] - closest_next_close)

            if distance_to_previous + distance_to_next != 0:
              if closest_previous_pivot_value == -1:
                proximity_values[i] = (distance_to_previous - distance_to_next) / (distance_to_previous + distance_to_next)
              else:
                proximity_values[i] = (distance_to_next - distance_to_previous) / (distance_to_previous + distance_to_next)

    df['Pivot_Proximity'] = proximity_values
    return df

In [17]:
data = calculate_pivot_proximity(data)
data

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity
Date,,,,,,,
2024-01-01 00:00:00,42283.58,42397.23,42261.02,42397.23,155.25731,0,0.0
2024-01-01 00:05:00,42397.22,42432.74,42385.26,42409.96,141.31102,0,0.0
2024-01-01 00:10:00,42409.97,42488.09,42409.96,42488.00,135.14249,0,0.0
2024-01-01 00:15:00,42488.00,42554.57,42458.58,42458.85,163.32541,0,0.0
2024-01-01 00:20:00,42458.85,42491.10,42445.84,42473.93,117.27112,0,0.0
...,...,...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035,0,0.0
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118,0,0.0
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156,0,0.0


In [18]:
def plot_candlestick_with_proximity(df, col='Pivot_Proximity', limit=0.0):
    # Create the candlestick trace
    candlestick = go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlesticks',
        increasing=dict(line=dict(color='green'), fillcolor='green'),
        decreasing=dict(line=dict(color='red'), fillcolor='red'),
    )

    # Create a bar trace for Pivot_Proximity coloring
    # Create a color list based on Pivot_Proximity
    colors = []
    for proximity in df[col]:
        if proximity > (0 + limit):
            colors.append('green')  # Positive proximity (closer to lower pivot)
        elif proximity < (0 + (-1 * limit)):
            colors.append('red')    # Negative proximity (closer to higher pivot)
        else:
            colors.append('gray')   # Neutral (middle or no pivot)
    proximity_trace = go.Bar(
        x=df.index,
        y=df['Close'],
        marker=dict(color=colors),
        opacity=0.3,
        name=col,
        yaxis='y2'
    )

    # Create pivot indicators
    higher_pivots_df = df[df[col] == -1]
    lower_pivots_df = df[df[col] == 1]

    higher_pivots = go.Scatter(
        x=higher_pivots_df.index,
        y=higher_pivots_df['High'],
        mode='markers',
        marker=dict(color='red', size=10, symbol='triangle-down'),
        name='Higher Pivot'
    )

    lower_pivots = go.Scatter(
        x=lower_pivots_df.index,
        y=lower_pivots_df['Low'],
        mode='markers',
        marker=dict(color='green', size=10, symbol='triangle-up'),
        name='Lower Pivot'
    )

    # Create the figure
    fig = go.Figure(data=[
        candlestick,
        higher_pivots,
        lower_pivots,
        proximity_trace
        ])

    # Update layout
    fig.update_layout(
        title='Candlestick Chart with Pivot Proximity',
        xaxis_title='Date',
        yaxis_title='Price',
        yaxis2=dict(title='Pivot Proximity', overlaying='y', side='right'),
        template='plotly_dark',
        showlegend=True,
        xaxis_rangeslider_visible=False,
    )

    # Show the figure
    fig.show()

In [19]:
plot_candlestick_with_proximity(data[0:500], 'Pivot_Proximity', 0.7)

In [20]:
data

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity
Date,,,,,,,
2024-01-01 00:00:00,42283.58,42397.23,42261.02,42397.23,155.25731,0,0.0
2024-01-01 00:05:00,42397.22,42432.74,42385.26,42409.96,141.31102,0,0.0
2024-01-01 00:10:00,42409.97,42488.09,42409.96,42488.00,135.14249,0,0.0
2024-01-01 00:15:00,42488.00,42554.57,42458.58,42458.85,163.32541,0,0.0
2024-01-01 00:20:00,42458.85,42491.10,42445.84,42473.93,117.27112,0,0.0
...,...,...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035,0,0.0
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118,0,0.0
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156,0,0.0


In [21]:
z = ta.bbands(data['Close'])
z

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,
2024-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:05:00,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:20:00,42374.119374,42445.594,42517.068626,0.336782,0.698224
...,...,...,...,...,...
2025-03-15 14:30:00,84064.565975,84221.732,84378.898025,0.373220,0.256302
2025-03-15 14:35:00,84098.972093,84181.342,84263.711907,0.195696,0.346230
2025-03-15 14:40:00,84129.576215,84170.032,84210.487785,0.096129,0.923648


In [22]:
z.columns[1]

'BBM_5_2.0'

In [23]:
features_columns = []

def add_scaled_rsi(df, window=14, prefix = ''):
  series = ta.rsi(df['Close'], length=window)
  column_name = f"{prefix}RSI_{window}"
  df[column_name] = series / 100
  if(column_name not in features_columns):
    features_columns.append(column_name)
  return df

def add_scaled_ema(df, window=50, prefix = ''):
  series = ta.ema(df['Close'], length=window)
  column_name = f"{prefix}EMA_{window}"
  df[column_name] = series / df['Close']
  if column_name not in features_columns:
    features_columns.append(column_name)
  return df

def add_scaled_sma(df, window=50, prefix = ''):
  series = ta.sma(df['Close'], length=window)
  column_name = f"{prefix}SMA_{window}"
  df[column_name] = series / df['Close']
  if column_name not in features_columns:
    features_columns.append(column_name)
  return df

def add_scaled_macd(df, prefix= '', fast = 12, slow = 26, signal=9):
  macd_df = ta.macd(df['Close'], fast=fast, slow=slow, signal=signal)
  suffix = f"{fast}_{slow}_{signal}"
  macd_column_name = f"{prefix}MACD_{suffix}"
  macds_column_name = f"{prefix}MACDs_{suffix}"
  macdh_column_name = f"{prefix}MACDh_{suffix}"
  df[macd_column_name] = macd_df[macd_df.columns[0]] / df['Close']
  df[macds_column_name] = macd_df[macd_df.columns[2]] / df['Close']
  df[macdh_column_name] = macd_df[macd_df.columns[1]] / df['Close']
  if macd_column_name not in features_columns:
    features_columns.append(macd_column_name)
  if macds_column_name not in features_columns:
    features_columns.append(macds_column_name)
  if macdh_column_name not in features_columns:
    features_columns.append(macdh_column_name)
  return df

def add_scaled_bbands(df, window=20, std=2.0, prefix= ''):
  bbands_df = ta.bbands(df['Close'], length=window, std=std)
  suffix = f"{window}_{std}"
  bbl_column_name = f"{prefix}BBL_{suffix}"
  bbm_column_name = f"{prefix}BBM_{suffix}"
  bbu_column_name = f"{prefix}BBU_{suffix}"
  bbb_column_name = f"{prefix}BBB_{suffix}"
  bbp_column_name = f"{prefix}BBP_{suffix}"
  df[bbl_column_name] = bbands_df[bbands_df.columns[0]] / df['Close']
  df[bbm_column_name] = bbands_df[bbands_df.columns[1]] / df['Close']
  df[bbu_column_name] = bbands_df[bbands_df.columns[2]] / df['Close']
  df[bbb_column_name] = bbands_df[bbands_df.columns[3]]
  df[bbp_column_name] = bbands_df[bbands_df.columns[4]]
  if bbl_column_name not in features_columns: features_columns.append(bbl_column_name)
  if bbm_column_name not in features_columns: features_columns.append(bbm_column_name)
  if bbu_column_name not in features_columns: features_columns.append(bbu_column_name)
  if bbb_column_name not in features_columns: features_columns.append(bbb_column_name)
  if bbp_column_name not in features_columns: features_columns.append(bbp_column_name)
  return df

In [24]:
# # Add technical indicators
lower_timeframe = timeframe
lower_timeframe_prefix = f"{lower_timeframe}_"
add_scaled_rsi(data, 14, prefix=lower_timeframe_prefix)
add_scaled_rsi(data, 6, prefix=lower_timeframe_prefix)
add_scaled_ema(data, 5, prefix=lower_timeframe_prefix)
add_scaled_ema(data, 21, prefix=lower_timeframe_prefix)
add_scaled_sma(data, 50, prefix=lower_timeframe_prefix)
add_scaled_macd(data, prefix=lower_timeframe_prefix)
add_scaled_bbands(data, prefix=lower_timeframe_prefix)
data

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,5m_EMA_21,5m_SMA_50,5m_MACD_12_26_9,5m_MACDs_12_26_9,5m_MACDh_12_26_9,5m_BBL_20_2.0,5m_BBM_20_2.0,5m_BBU_20_2.0,5m_BBB_20_2.0,5m_BBP_20_2.0
Date,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,42283.58,42397.23,42261.02,42397.23,155.25731,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:05:00,42397.22,42432.74,42385.26,42409.96,141.31102,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:10:00,42409.97,42488.09,42409.96,42488.00,135.14249,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:15:00,42488.00,42554.57,42458.58,42458.85,163.32541,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:20:00,42458.85,42491.10,42445.84,42473.93,117.27112,0,0.0,NaN,NaN,0.999333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035,0,0.0,0.433494,0.310195,1.000737,1.001412,1.000004,0.000146,0.000596,-0.000450,0.999561,1.001941,1.004321,0.475041,0.092180
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118,0,0.0,0.441287,0.334064,1.000405,1.001166,0.999978,0.000033,0.000483,-0.000451,0.999482,1.001823,1.004165,0.467468,0.110712
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156,0,0.0,0.475762,0.437712,0.999887,1.000538,0.999496,-0.000011,0.000384,-0.000395,0.998831,1.001219,1.003606,0.476933,0.244818


In [25]:
features_columns

['5m_RSI_14',
 '5m_RSI_6',
 '5m_EMA_5',
 '5m_EMA_21',
 '5m_SMA_50',
 '5m_MACD_12_26_9',
 '5m_MACDs_12_26_9',
 '5m_MACDh_12_26_9',
 '5m_BBL_20_2.0',
 '5m_BBM_20_2.0',
 '5m_BBU_20_2.0',
 '5m_BBB_20_2.0',
 '5m_BBP_20_2.0']

In [26]:
def resample_candles(df, interval='1h'):
    """
    Resamples the DataFrame to the specified interval.
    If the interval is in the format '15m', it's converted to '15min'.
    """
    if interval.endswith('m') and not interval.endswith('min'):
        interval = interval[:-1] + 'min'  # Convert '15m' to '15min'

    return df.resample(interval).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    }).ffill()

In [27]:
middle_timeframe = '15m'
middle_timeframe_prefix = f"{middle_timeframe}_"
middle_data = resample_candles(data, middle_timeframe)
add_scaled_rsi(middle_data, 14, prefix=middle_timeframe_prefix)
add_scaled_rsi(middle_data, 6, prefix=middle_timeframe_prefix)
add_scaled_ema(middle_data, 5, prefix=middle_timeframe_prefix)
add_scaled_ema(middle_data, 21, prefix=middle_timeframe_prefix)
add_scaled_macd(middle_data, prefix=middle_timeframe_prefix)
add_scaled_bbands(middle_data, prefix=middle_timeframe_prefix)
middle_data

,Open,High,Low,Close,Volume,15m_RSI_14,15m_RSI_6,15m_EMA_5,15m_EMA_21,15m_MACD_12_26_9,15m_MACDs_12_26_9,15m_MACDh_12_26_9,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0
Date,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,42283.58,42488.09,42261.02,42488.00,431.71082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:15:00,42488.00,42554.57,42412.02,42419.73,392.24889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:30:00,42419.73,42447.82,42354.19,42441.32,319.90644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:45:00,42441.32,42490.74,42422.45,42475.23,127.81493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 01:00:00,42475.23,42475.23,42431.65,42466.33,188.76099,NaN,NaN,0.999807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-15 13:45:00,84460.79,84499.74,84362.15,84460.00,217.91395,0.659938,0.822339,0.998787,0.996425,0.000979,0.000248,0.000731,0.990153,0.995248,1.000342,1.023745,0.966444
2025-03-15 14:00:00,84459.99,84459.99,84218.43,84357.96,184.73100,0.598261,0.632167,0.999997,0.997846,0.001031,0.000404,0.000626,0.991633,0.996774,1.001915,1.031486,0.813752
2025-03-15 14:15:00,84357.97,84357.97,84134.63,84177.01,235.94096,0.507655,0.423672,1.001431,0.999992,0.000889,0.000502,0.000387,0.994344,0.999171,1.003999,0.966355,0.585827


In [28]:
# add 4h data
higher_timeframe = '1h'
higher_timeframe_prefix = f"{higher_timeframe}_"
higher_data = resample_candles(data, higher_timeframe)
add_scaled_rsi(higher_data, 14, prefix=higher_timeframe_prefix)
add_scaled_rsi(higher_data, 6, prefix=higher_timeframe_prefix)
add_scaled_ema(higher_data, 5, prefix=higher_timeframe_prefix)
add_scaled_ema(higher_data, 21, prefix=higher_timeframe_prefix)
higher_data

,Open,High,Low,Close,Volume,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21
Date,,,,,,,,,
2024-01-01 00:00:00,42283.58,42554.57,42261.02,42475.23,1271.68108,NaN,NaN,NaN,NaN
2024-01-01 01:00:00,42475.23,42775.00,42431.65,42613.56,1196.37856,NaN,NaN,NaN,NaN
2024-01-01 02:00:00,42613.57,42638.41,42500.00,42581.10,685.21980,NaN,NaN,NaN,NaN
2024-01-01 03:00:00,42581.09,42586.64,42230.08,42330.49,794.80391,NaN,NaN,NaN,NaN
2024-01-01 04:00:00,42330.50,42399.99,42209.46,42399.99,715.41760,NaN,NaN,1.001889,NaN
...,...,...,...,...,...,...,...,...,...
2025-03-15 10:00:00,83966.95,83994.11,83667.23,83912.19,385.90322,0.519255,0.382626,1.000945,1.000368
2025-03-15 11:00:00,83912.19,84184.58,83842.95,84103.74,436.48745,0.551314,0.513349,0.999110,0.998264
2025-03-15 12:00:00,84103.74,84272.19,84010.58,84136.53,638.42673,0.556763,0.533634,0.999147,0.998068


In [29]:
features_columns

['5m_RSI_14',
 '5m_RSI_6',
 '5m_EMA_5',
 '5m_EMA_21',
 '5m_SMA_50',
 '5m_MACD_12_26_9',
 '5m_MACDs_12_26_9',
 '5m_MACDh_12_26_9',
 '5m_BBL_20_2.0',
 '5m_BBM_20_2.0',
 '5m_BBU_20_2.0',
 '5m_BBB_20_2.0',
 '5m_BBP_20_2.0',
 '15m_RSI_14',
 '15m_RSI_6',
 '15m_EMA_5',
 '15m_EMA_21',
 '15m_MACD_12_26_9',
 '15m_MACDs_12_26_9',
 '15m_MACDh_12_26_9',
 '15m_BBL_20_2.0',
 '15m_BBM_20_2.0',
 '15m_BBU_20_2.0',
 '15m_BBB_20_2.0',
 '15m_BBP_20_2.0',
 '1h_RSI_14',
 '1h_RSI_6',
 '1h_EMA_5',
 '1h_EMA_21']

In [30]:
def merge_candlesticks_data(df1, df2):
  """
  Merges two candlestick DataFrames with forward fill, handling different timeframes,
  and prevents duplicate OHLCV columns.
  Ensure both DataFrames have a datetime index.
  And ensure the second DataFrame is the larger timeframe

  Args:
      df1: First candlestick DataFrame with datetime index.
      df2: Second candlestick DataFrame with datetime index.

  Returns:
      Merged DataFrame with forward-filled values, and no duplicate OHLCV columns.
  """
  # Ensure both DataFrames have a datetime index
  if not isinstance(df1.index, pd.DatetimeIndex) or not isinstance(df2.index, pd.DatetimeIndex):
      raise ValueError("DataFrames must have a datetime index.")
  # Identify OHLCV columns
  ohlcv_cols = ['open', 'high', 'low', 'close', 'volume']

  # Rename columns in df2 that conflict with df1's OHLCV columns
  for col in df2.columns:
    if col.lower() in ohlcv_cols and col.lower() in df1.columns.str.lower():
      del df2[col]

  # Merge the DataFrames using outer join, which preserves all dates
  merged_df = pd.merge(df1, df2, how='outer', left_index=True, right_index=True, suffixes=('_df1', '_df2'))

  # Forward fill the missing values for each column
  for col in merged_df.columns:
    merged_df[col] = merged_df[col].ffill()

  return merged_df

In [31]:
data = merge_candlesticks_data(data, middle_data)
data = merge_candlesticks_data(data, higher_data)
data

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,...,15m_MACDh_12_26_9,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,42283.58,42397.23,42261.02,42397.23,155.25731,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:05:00,42397.22,42432.74,42385.26,42409.96,141.31102,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:10:00,42409.97,42488.09,42409.96,42488.00,135.14249,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:15:00,42488.00,42554.57,42458.58,42458.85,163.32541,0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01 00:20:00,42458.85,42491.10,42445.84,42473.93,117.27112,0,0.0,NaN,NaN,0.999333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035,0,0.0,0.433494,0.310195,1.000737,...,0.000232,0.994296,0.999022,1.003748,0.946179,0.603449,0.553862,0.524377,0.999926,0.997973
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118,0,0.0,0.441287,0.334064,1.000405,...,0.000232,0.994296,0.999022,1.003748,0.946179,0.603449,0.553862,0.524377,0.999926,0.997973
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156,0,0.0,0.475762,0.437712,0.999887,...,0.000232,0.994296,0.999022,1.003748,0.946179,0.603449,0.553862,0.524377,0.999926,0.997973


In [32]:
# Drop rows with NaN values (due to rolling calculations)
data.dropna(inplace=True)
data

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,...,15m_MACDh_12_26_9,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 20:00:00,43517.98,43686.63,43510.71,43686.63,576.96245,0,-0.856429,0.891703,0.957965,0.996507,...,0.001611,0.972047,0.985081,0.998115,2.646304,1.072297,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:05:00,43686.62,43700.00,43618.01,43692.54,242.68469,0,-0.867686,0.892774,0.958786,0.997582,...,0.001611,0.972047,0.985081,0.998115,2.646304,1.072297,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:10:00,43692.54,43757.13,43643.98,43643.98,352.27988,0,-0.775185,0.820924,0.804065,0.999128,...,0.001611,0.972047,0.985081,0.998115,2.646304,1.072297,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:15:00,43643.99,43703.96,43589.00,43604.98,259.19851,0,-0.700894,0.767501,0.695846,1.000014,...,0.001712,0.970628,0.984991,0.999355,2.916464,1.022464,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:20:00,43604.99,43641.72,43559.94,43638.71,192.90856,0,-0.765146,0.780788,0.733124,0.999494,...,0.001712,0.970628,0.984991,0.999355,2.916464,1.022464,0.826199,0.904883,0.990741,0.977408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-15 14:30:00,84177.02,84228.92,84145.12,84145.13,58.00035,0,0.000000,0.433494,0.310195,1.000737,...,0.000232,0.994296,0.999022,1.003748,0.946179,0.603449,0.553862,0.524377,0.999926,0.997973
2025-03-15 14:35:00,84145.13,84185.15,84106.83,84156.01,105.87118,0,0.000000,0.441287,0.334064,1.000405,...,0.000232,0.994296,0.999022,1.003748,0.946179,0.603449,0.553862,0.524377,0.999926,0.997973
2025-03-15 14:40:00,84156.01,84204.32,84156.01,84204.31,92.16156,0,0.000000,0.475762,0.437712,0.999887,...,0.000232,0.994296,0.999022,1.003748,0.946179,0.603449,0.553862,0.524377,0.999926,0.997973


In [33]:
# delete Pivot_Porximity that have zero at the beginning and ending of the dataframe
non_zero_indices = data[data['Pivot_Proximity'] != 0].index
non_zero_at_begining = non_zero_indices[0]
non_zero_at_end = non_zero_indices[-1]
data = data.loc[non_zero_at_begining:non_zero_at_end]
data

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,...,15m_MACDh_12_26_9,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-01 20:00:00,43517.98,43686.63,43510.71,43686.63,576.96245,0,-0.856429,0.891703,0.957965,0.996507,...,0.001611,0.972047,0.985081,0.998115,2.646304,1.072297,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:05:00,43686.62,43700.00,43618.01,43692.54,242.68469,0,-0.867686,0.892774,0.958786,0.997582,...,0.001611,0.972047,0.985081,0.998115,2.646304,1.072297,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:10:00,43692.54,43757.13,43643.98,43643.98,352.27988,0,-0.775185,0.820924,0.804065,0.999128,...,0.001611,0.972047,0.985081,0.998115,2.646304,1.072297,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:15:00,43643.99,43703.96,43589.00,43604.98,259.19851,0,-0.700894,0.767501,0.695846,1.000014,...,0.001712,0.970628,0.984991,0.999355,2.916464,1.022464,0.826199,0.904883,0.990741,0.977408
2024-01-01 20:20:00,43604.99,43641.72,43559.94,43638.71,192.90856,0,-0.765146,0.780788,0.733124,0.999494,...,0.001712,0.970628,0.984991,0.999355,2.916464,1.022464,0.826199,0.904883,0.990741,0.977408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-15 13:30:00,84301.89,84380.00,84235.45,84369.91,99.21654,0,-0.779255,0.633500,0.667683,0.999496,...,0.000712,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768
2025-03-15 13:35:00,84369.91,84427.16,84353.88,84404.86,101.39651,0,-0.875034,0.651155,0.706400,0.999388,...,0.000712,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768
2025-03-15 13:40:00,84404.87,84470.74,84404.86,84460.79,41.96319,0,-0.972470,0.677897,0.760078,0.999151,...,0.000712,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768


In [34]:
data[features_columns].describe()

,5m_RSI_14,5m_RSI_6,5m_EMA_5,5m_EMA_21,5m_SMA_50,5m_MACD_12_26_9,5m_MACDs_12_26_9,5m_MACDh_12_26_9,5m_BBL_20_2.0,5m_BBM_20_2.0,...,15m_MACDh_12_26_9,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21
count,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000,1.263590e+05,126359.000000,126359.000000,...,1.263590e+05,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000,126359.000000
mean,0.504695,0.504682,0.999992,0.999961,0.999903,0.000028,0.000028,-1.099748e-07,0.995388,0.999963,...,-3.576099e-07,0.991904,0.999887,1.007871,1.596672,0.512781,0.511568,0.512962,0.999905,0.999511
std,0.108991,0.168805,0.001495,0.003605,0.006597,0.001864,0.001755,5.601455e-04,0.005556,0.004111,...,9.600894e-04,0.009470,0.007058,0.009934,1.330623,0.330252,0.119988,0.172996,0.005081,0.012181
min,0.063100,0.009524,0.976288,0.955989,0.930748,-0.021544,-0.019054,-8.432739e-03,0.908063,0.948035,...,-9.334411e-03,0.867522,0.928312,0.978274,0.098702,-0.528913,0.124816,0.030257,0.956054,0.925080
25%,0.433073,0.384162,0.999358,0.998483,0.997106,-0.000694,-0.000657,-2.468996e-04,0.994021,0.998264,...,-4.274980e-04,0.989533,0.996883,1.002314,0.782833,0.259958,0.434099,0.396137,0.997762,0.993669
50%,0.505800,0.505839,0.999985,0.999907,0.999796,0.000061,0.000058,-4.364393e-06,0.996882,0.999910,...,-1.620716e-06,0.994508,0.999774,1.005020,1.195304,0.526715,0.512686,0.516223,0.999801,0.999240
75%,0.575892,0.625210,1.000593,1.001326,1.002464,0.000818,0.000778,2.463785e-04,0.998615,1.001567,...,4.417967e-04,0.997477,1.002656,1.009871,1.935229,0.765425,0.588220,0.631238,1.001881,1.004917
max,0.957961,0.993054,1.046445,1.080406,1.098670,0.017601,0.016449,6.515587e-03,1.038605,1.086729,...,9.552534e-03,1.033378,1.078095,1.131329,15.228677,1.531520,0.901537,0.973768,1.049538,1.098367


In [35]:
def create_sequences(data, features_columns, target_col, seq_length):
    """
    Create sequences of `seq_length` time steps for LSTM input, optimized for performance.
    """
    num_samples = len(data) - seq_length
    num_features = len(features_columns)

    X = np.zeros((num_samples, seq_length, num_features))
    y = np.zeros(num_samples)

    features_data = data[features_columns].values
    target_data = data[target_col].values

    for i in range(num_samples):
        X[i] = features_data[i:i + seq_length]
        y[i] = target_data[i + seq_length]

    return X, y

In [36]:
def calc_class_ratios(df, column):
  class_counts = df[column].value_counts()
  total_count = len(df)
  class_ratios = class_counts / total_count
  return class_ratios.to_dict()

def class_ratio_to_class_weights(class_ratios):
  class_weights = {}
  for key, value in class_ratios.items():
    class_weights[key] = 1 / value
  return class_weights

class_ratios = calc_class_ratios(data, 'Pivot')
class_weights = class_ratio_to_class_weights(class_ratios)
class_weights

{0: 1.0579640979269231, 1: 36.498844598497975, -1: 36.50939034960994}

In [37]:
# select only first 80 % of the data
training_data_ratio = 0.80 # 0.80 for 80% during testing and reserach, 0.99999 for Prod
training_data = data[:int(len(data) * training_data_ratio)]
training_data_start_date = training_data.index[0]
training_data_end_date = training_data.index[-1]
training_data.tail(10)

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,...,15m_MACDh_12_26_9,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-17 19:05:00,106656.00,106751.20,106334.45,106499.99,145.28045,0,0.709073,0.377895,0.294363,1.001394,...,-0.000345,0.992820,1.002614,1.012408,1.953747,0.366546,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:10:00,106499.99,106810.05,106499.99,106810.05,85.72708,0,0.208019,0.457962,0.481683,0.998991,...,-0.000345,0.992820,1.002614,1.012408,1.953747,0.366546,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:15:00,106810.04,106810.04,106574.11,106699.32,131.01293,0,0.386957,0.436363,0.432482,1.000019,...,-0.000492,0.993899,1.003442,1.012984,1.901989,0.319670,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:20:00,106699.32,106704.00,106486.79,106692.18,87.75292,0,0.398496,0.434938,0.429091,1.000057,...,-0.000492,0.993899,1.003442,1.012984,1.901989,0.319670,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:25:00,106692.19,106986.44,106626.16,106669.07,183.27127,0,0.435841,0.430045,0.416408,1.000183,...,-0.000492,0.993899,1.003442,1.012984,1.901989,0.319670,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:30:00,106669.07,106669.08,106505.47,106507.17,51.32480,0,0.697470,0.396398,0.333533,1.001135,...,-0.000747,0.996331,1.005729,1.015127,1.868908,0.195194,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:35:00,106507.16,106507.16,106314.74,106360.63,98.57593,0,0.934278,0.368309,0.274248,1.001676,...,-0.000747,0.996331,1.005729,1.015127,1.868908,0.195194,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:40:00,106360.64,106381.01,106216.06,106355.85,140.02351,0,0.942002,0.367394,0.272353,1.001148,...,-0.000747,0.996331,1.005729,1.015127,1.868908,0.195194,0.495243,0.417581,1.003423,1.002395
2024-12-17 19:45:00,106355.84,106448.28,106217.44,106393.22,148.84533,0,0.881612,0.380348,0.316649,1.000531,...,-0.000793,0.996010,1.003758,1.011507,1.543942,0.257490,0.495243,0.417581,1.003423,1.002395


In [38]:
seq_length = 500
target_col = 'Pivot_Proximity'

# Create sequences
X, y = create_sequences(training_data, features_columns, target_col, seq_length)

In [54]:
print(X.shape)
print(y.shape)

(100587, 500, 29)
(100587,)


In [40]:
# Split into training and testing sets
train_validate_split_ratio = 0.2

In [43]:
# Use GPU if available
# Check if a GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to use the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU found.")

1 Physical GPUs, 1 Logical GPU


In [46]:
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=2):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience  # Number of consecutive increases allowed
        self.best_val_mae = float('inf')  # Track the best validation MAE
        self.increase_count = 0  # Counter for consecutive increases

    def on_epoch_end(self, epoch, logs=None):
        current_val_mae = logs.get('val_mae')  # Get validation MAE for the current epoch

        if current_val_mae < self.best_val_mae:
            # If validation MAE improves, reset the counter
            self.best_val_mae = current_val_mae
            self.increase_count = 0
        else:
            # If validation MAE increases, increment the counter
            self.increase_count += 1

        # Stop training if validation MAE increases consecutively for 'patience' epochs
        if self.increase_count >= self.patience:
            print(f"\nEarly stopping: Validation MAE increased {self.patience} times in a row.")
            self.model.stop_training = True

In [55]:
def train_in_chunks(X, y, chunk_size=100000, train_validate_split_ratio=0.2, train_epochs=20):
    """Retrains the model in chunks to handle memory limit issues."""

    num_chunks = int(np.ceil(len(X) / chunk_size))
    model = None  # Initialize model outside the loop
    initial_learning_rate = 0.001  # Initial learning rate
    learning_rate_decay = 0.9     # Decay factor (adjust as needed)

    if num_chunks > 1 and len(X) % chunk_size < 0.6 * chunk_size:
        num_chunks -= 1  # Combine last two chunks

    for chunk_idx in range(num_chunks):
        print(f"Processing chunk {chunk_idx + 1}/{num_chunks}")
        checkpoint_filepath = 'weights-' + str(chunk_idx) + '.{epoch:02d}-{val_loss:.4f}.keras'
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
          filepath=checkpoint_filepath,
          save_best_only=True,
          monitor='val_loss',
          mode='min'
      )
        start_idx = chunk_idx * chunk_size
        end_idx = min((chunk_idx + 1) * chunk_size, len(X))

        X_chunk = X[start_idx:end_idx]
        y_chunk = y[start_idx:end_idx]

        X_train, X_test, y_train, y_test = train_test_split(
            X_chunk, y_chunk, test_size=train_validate_split_ratio, shuffle=False
        )

        model_input_shape = (X_train.shape[1], X_train.shape[2])

        history = None

        if model is None:
            # Build the model only for the first chunk
            model = Sequential([
                LSTM(100, return_sequences=True, input_shape=model_input_shape),
                Dropout(0.2),
                LSTM(50, return_sequences=False),
                Dropout(0.2),
                Dense(50, activation='relu'),
                Dropout(0.2),
                Dense(25, activation='relu'),
                Dropout(0.2),
                Dense(10, activation='relu'),
                Dropout(0.2),
                Dense(1, activation='tanh')
            ])
            optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
            model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        else:
            # Reduce learning rate for subsequent chunks
            current_learning_rate = tf.keras.backend.get_value(model.optimizer.learning_rate)
            new_learning_rate = current_learning_rate * learning_rate_decay
            # new_learning_rate = float(new_learning_rate)
            # tf.keras.backend.set_value(model.optimizer.learning_rate, new_learning_rate)
            model.optimizer.learning_rate.assign(new_learning_rate)
            print(f"Learning rate reduced to: {new_learning_rate}")

        custom_early_stopping = CustomEarlyStopping(patience=3)

        history = model.fit(
            X_train, y_train,
            epochs=train_epochs,
            batch_size=32,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[custom_early_stopping, model_checkpoint_callback]
        )

        final_epoch = len(history.history['loss'])
        train_loss = history.history['loss'][-1]
        train_mae = history.history['mae'][-1]
        val_loss = history.history['val_loss'][-1]
        val_mae = history.history['val_mae'][-1]

        # Get the best val_loss
        best_val_loss = min(history.history['val_loss'])
        best_epoch = history.history['val_loss'].index(best_val_loss) + 1

        print(f"Best Epoch: {best_epoch}")
        print(f"Best Val Loss: {best_val_loss}")

        # Load the model of the best
        model.load_weights(checkpoint_filepath.format(epoch=best_epoch, val_loss=best_val_loss))

        print(f"Chunk {chunk_idx + 1} training complete.")
        print(f"Final Epoch: {final_epoch}")
        print(f"Train Loss: {train_loss}")
        print(f"Train MAE: {train_mae}")
        print(f"Val Loss: {val_loss}")
        print(f"Val MAE: {val_mae}")
        gc.collect() #garbage collection to free memory.
        tf.keras.backend.clear_session()#clear session.

    # Delete all weight files
    for filename in os.listdir('.'):
        if filename.startswith('weights-') and filename.endswith('.keras'):
            os.remove(filename)
    return model, history

In [57]:
model, history = train_in_chunks(
    X, y, chunk_size=50000,
    train_validate_split_ratio=train_validate_split_ratio
)

Processing chunk 1/2
Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 49s 37ms/step - loss: 0.3383 - mae: 0.4990 - val_loss: 0.2753 - val_mae: 0.4467
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 0.2745 - mae: 0.4417 - val_loss: 0.2488 - val_mae: 0.4223
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 36ms/step - loss: 0.2640 - mae: 0.4304 - val_loss: 0.2387 - val_mae: 0.4118
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 0.2539 - mae: 0.4194 - val_loss: 0.2322 - val_mae: 0.4025
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 0.2481 - mae: 0.4144 - val_loss: 0.2243 - val_mae: 0.3941
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 36ms/step - loss: 0.2465 - mae: 0.4112 - val_loss: 0.2287 - val_mae: 0.4009
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 35ms/step - loss: 0.2391 - mae: 0.4048 - val_loss: 0.2320 - val_mae: 0.4021
Epoch 8/20
1249/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2389 - mae: 0.4033
Early stopping: Validation MAE increased 

In [58]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 500, 100)            │          52,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 500, 100)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          30,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │           2,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 258,890 (1011.29 KB)

 Trainable params: 86,296 (337.09 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 172,594 (674.20 KB)

In [59]:
def generate_random_hash(max_length=8):
    """
    Generate a random hash of a maximum length.

    Args:
        max_length (int): Maximum length of the hash (default is 8).

    Returns:
        str: A random hash of the specified maximum length.
    """
    # Generate a random byte sequence
    random_bytes = os.urandom(16)  # 16 bytes of random data
    # Create a SHA-256 hash of the random bytes
    hash_object = hashlib.sha256(random_bytes)
    # Get the hexadecimal representation of the hash
    hex_hash = hash_object.hexdigest()
    # Truncate the hash to the desired maximum length
    return hex_hash[:max_length]

In [ ]:
# Delete the model and free resources
# del model
# K.clear_session()
# gc.collect()

In [ ]:
# Load a model that has a good loss
# model = tf.keras.models.load_model('weights.05-0.12.keras')

In [60]:
final_epoch = len(history.history['loss'])
print(f"Final Epoch: {final_epoch}")

Final Epoch: 4


In [61]:
train_loss = history.history['loss'][-1]
train_mae = history.history['mae'][-1]
val_loss = history.history['val_loss'][-1]
val_mae = history.history['val_mae'][-1]
print(f"Train Loss: {train_loss:.4f}")
print(f"Train MAE: {train_mae:.4f}")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation MAE: {val_mae:.4f}")

Train Loss: 0.2322
Train MAE: 0.3973
Validation Loss: 0.2134
Validation MAE: 0.3821


In [62]:
px.line(history.history, y=['loss', 'val_loss'], title='Loss Over Epochs')

In [63]:
px.line(history.history, y=['mae', 'val_mae'], title='MAE Over Epochs')

In [64]:
test_data = data[int(len(data) * training_data_ratio):]
test_data.tail(10)

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,...,15m_MACDh_12_26_9,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21
Date,,,,,,,,,,,,,,,,,,,,,
2025-03-15 13:05:00,84254.72,84285.11,84254.71,84284.19,52.44428,0,-0.544341,0.612424,0.644965,0.999325,...,0.000616,0.992566,0.996382,1.000197,0.765835,0.974192,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:10:00,84284.19,84309.35,84262.81,84288.40,25.21437,0,-0.555878,0.614623,0.649830,0.999517,...,0.000616,0.992566,0.996382,1.000197,0.765835,0.974192,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:15:00,84288.41,84358.56,84254.71,84341.68,58.98992,0,-0.701891,0.642283,0.710141,0.999257,...,0.000621,0.992462,0.996506,1.000551,0.811671,0.931924,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:20:00,84341.68,84366.45,84341.68,84366.45,47.97298,0,-0.769773,0.654692,0.735551,0.999309,...,0.000621,0.992462,0.996506,1.000551,0.811671,0.931924,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:25:00,84366.45,84384.55,84266.00,84301.89,72.08014,0,-0.592847,0.596603,0.577274,1.000050,...,0.000621,0.992462,0.996506,1.000551,0.811671,0.931924,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:30:00,84301.89,84380.00,84235.45,84369.91,99.21654,0,-0.779255,0.633500,0.667683,0.999496,...,0.000712,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:35:00,84369.91,84427.16,84353.88,84404.86,101.39651,0,-0.875034,0.651155,0.706400,0.999388,...,0.000712,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:40:00,84404.87,84470.74,84404.86,84460.79,41.96319,0,-0.972470,0.677897,0.760078,0.999151,...,0.000712,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.99688,0.994768
2025-03-15 13:45:00,84460.79,84460.79,84392.54,84403.92,40.65772,0,-0.872458,0.625400,0.621446,0.999883,...,0.000731,0.990153,0.995248,1.000342,1.023745,0.966444,0.607413,0.687722,0.99688,0.994768


In [65]:
def predict_in_batches(model, test_data, features_columns, seq_length, batch_size=128):
    """
    Predicts in batches to speed up inference.
    """
    num_samples = len(test_data) - seq_length
    feature_array = test_data[features_columns].values
    predicted_values = np.full(len(test_data), np.nan)

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = range(start_idx + seq_length, end_idx + seq_length)
        batch_input = np.array([feature_array[i - seq_length:i] for i in batch_indices])

        if len(batch_input) > 0 :
            predictions = model.predict(batch_input, verbose=0)
            predicted_values[batch_indices] = predictions.flatten() #Flatten to 1d array.

    test_data['Predicted_Value'] = predicted_values
    return test_data

In [66]:
# Predict in batches
batch_size = 1024
predict_in_batches(model, test_data, features_columns, seq_length, batch_size)

<ipython-input-65-54e86c8c4e9a>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,...,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21,Predicted_Value
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-17 19:55:00,106495.68,106554.99,106452.02,106475.89,69.85664,0,0.748018,0.410807,0.414641,0.999932,...,0.996010,1.003758,1.011507,1.543942,0.257490,0.495243,0.417581,1.003423,1.002395,NaN
2024-12-17 20:00:00,106475.89,106607.90,106448.84,106556.00,88.84483,0,0.618561,0.439448,0.504302,0.999453,...,0.997550,1.005744,1.013939,1.629592,0.149510,0.515097,0.466816,1.000671,0.999982,NaN
2024-12-17 20:05:00,106555.99,106556.00,106200.00,106284.15,118.05558,0,0.945297,0.373157,0.310580,1.001340,...,0.997550,1.005744,1.013939,1.629592,0.149510,0.515097,0.466816,1.000671,0.999982,NaN
2024-12-17 20:10:00,106284.14,106328.60,106165.41,106229.60,77.01948,0,0.872584,0.361376,0.284285,1.001236,...,0.997550,1.005744,1.013939,1.629592,0.149510,0.515097,0.466816,1.000671,0.999982,NaN
2024-12-17 20:15:00,106229.61,106331.16,106156.22,106319.96,102.45973,1,1.000000,0.395424,0.387386,1.000257,...,0.996793,1.004499,1.012206,1.534440,0.208096,0.515097,0.466816,1.000671,0.999982,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-15 13:30:00,84301.89,84380.00,84235.45,84369.91,99.21654,0,-0.779255,0.633500,0.667683,0.999496,...,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768,-0.250226
2025-03-15 13:35:00,84369.91,84427.16,84353.88,84404.86,101.39651,0,-0.875034,0.651155,0.706400,0.999388,...,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768,-0.404742
2025-03-15 13:40:00,84404.87,84470.74,84404.86,84460.79,41.96319,0,-0.972470,0.677897,0.760078,0.999151,...,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768,-0.490293


In [67]:
test_data[['Open', 'High', 'Low', 'Close', 'Pivot', 'Predicted_Value']]

,Open,High,Low,Close,Pivot,Predicted_Value
Date,,,,,,
2024-12-17 19:55:00,106495.68,106554.99,106452.02,106475.89,0,NaN
2024-12-17 20:00:00,106475.89,106607.90,106448.84,106556.00,0,NaN
2024-12-17 20:05:00,106555.99,106556.00,106200.00,106284.15,0,NaN
2024-12-17 20:10:00,106284.14,106328.60,106165.41,106229.60,0,NaN
2024-12-17 20:15:00,106229.61,106331.16,106156.22,106319.96,1,NaN
...,...,...,...,...,...,...
2025-03-15 13:30:00,84301.89,84380.00,84235.45,84369.91,0,-0.250226
2025-03-15 13:35:00,84369.91,84427.16,84353.88,84404.86,0,-0.404742
2025-03-15 13:40:00,84404.87,84470.74,84404.86,84460.79,0,-0.490293


In [68]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25272 entries, 2024-12-17 19:55:00 to 2025-03-15 13:50:00
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Open               25272 non-null  float64
 1   High               25272 non-null  float64
 2   Low                25272 non-null  float64
 3   Close              25272 non-null  float64
 4   Volume             25272 non-null  float64
 5   Pivot              25272 non-null  int64  
 6   Pivot_Proximity    25272 non-null  float64
 7   5m_RSI_14          25272 non-null  float64
 8   5m_RSI_6           25272 non-null  float64
 9   5m_EMA_5           25272 non-null  float64
 10  5m_EMA_21          25272 non-null  float64
 11  5m_SMA_50          25272 non-null  float64
 12  5m_MACD_12_26_9    25272 non-null  float64
 13  5m_MACDs_12_26_9   25272 non-null  float64
 14  5m_MACDh_12_26_9   25272 non-null  float64
 15  5m_BBL_20_2.0      25272 non-null  

In [69]:
test_data['Predicted_Value']

,Predicted_Value
Date,
2024-12-17 19:55:00,NaN
2024-12-17 20:00:00,NaN
2024-12-17 20:05:00,NaN
2024-12-17 20:10:00,NaN
2024-12-17 20:15:00,NaN
...,...
2025-03-15 13:30:00,-0.250226
2025-03-15 13:35:00,-0.404742
2025-03-15 13:40:00,-0.490293


In [70]:
test_data.dropna(inplace=True)
test_data

<ipython-input-70-686b255c5e96>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,5m_RSI_14,5m_RSI_6,5m_EMA_5,...,15m_BBL_20_2.0,15m_BBM_20_2.0,15m_BBU_20_2.0,15m_BBB_20_2.0,15m_BBP_20_2.0,1h_RSI_14,1h_RSI_6,1h_EMA_5,1h_EMA_21,Predicted_Value
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-19 13:35:00,102001.92,102001.93,101670.00,101771.98,189.09049,0,0.897320,0.349380,0.239401,1.002062,...,0.997135,1.002756,1.008376,1.121028,0.254844,0.465247,0.534156,1.000409,1.001378,0.340173
2024-12-19 13:40:00,101771.98,101843.99,101667.00,101813.76,188.15150,1,1.000000,0.371594,0.293770,1.001100,...,0.997135,1.002756,1.008376,1.121028,0.254844,0.465247,0.534156,1.000409,1.001378,0.582869
2024-12-19 13:45:00,101813.76,102053.59,101812.00,102051.99,245.34508,0,-0.151983,0.480506,0.525739,0.999176,...,0.996269,1.001545,1.006821,1.053530,0.353588,0.465247,0.534156,1.000409,1.001378,0.511007
2024-12-19 13:50:00,102051.99,102053.60,101914.68,101914.68,173.05917,0,0.511992,0.433835,0.428412,1.000348,...,0.996269,1.001545,1.006821,1.053530,0.353588,0.465247,0.534156,1.000409,1.001378,0.251771
2024-12-19 13:55:00,101914.67,101956.48,101864.70,101952.44,82.25910,0,0.329400,0.449665,0.461320,0.999985,...,0.996269,1.001545,1.006821,1.053530,0.353588,0.465247,0.534156,1.000409,1.001378,0.298272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-15 13:30:00,84301.89,84380.00,84235.45,84369.91,99.21654,0,-0.779255,0.633500,0.667683,0.999496,...,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768,-0.250226
2025-03-15 13:35:00,84369.91,84427.16,84353.88,84404.86,101.39651,0,-0.875034,0.651155,0.706400,0.999388,...,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768,-0.404742
2025-03-15 13:40:00,84404.87,84470.74,84404.86,84460.79,41.96319,0,-0.972470,0.677897,0.760078,0.999151,...,0.990236,0.994902,0.999567,0.937866,1.046407,0.607413,0.687722,0.996880,0.994768,-0.490293


In [71]:
plot_candlestick_with_proximity(test_data, 'Predicted_Value', 0.5)

In [72]:
test_data_start_date = test_data.index[0]
test_data_end_date = test_data.index[-1]
print(test_data_start_date)
print(test_data_end_date)

2024-12-19 13:35:00
2025-03-15 13:50:00


In [73]:
class PredictedValueStrategy(bt.Strategy):
    params = (
        ('buy_threshold', 0.4),
        ('sell_threshold', -0.4),
        ('leverage', 1),  # Leverage ratio
        ('margin', 1000)
    )

    def __init__(self):
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add the Predicted_Value as a data feed
        self.predicted_value = self.datas[0].predicted_value

         # Set leverage
        # self.broker.set_leverage(self.params.leverage)

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # # Check if we are in the market
        # if not self.position:
        #   # Buy signal
        #   if self.predicted_value[0] > self.params.buy_threshold:
        #     self.log('BUY CREATE, %.2f' % self.datas[0].close[0])
        #     self.order = self.buy()
        # else:
        #   # Sell signal
        #   if self.predicted_value[0] < self.params.sell_threshold:
        #     self.log('SELL CREATE, %.2f' % self.datas[0].close[0])
        #     self.order = self.sell()



         # # Check if we are in the market
        if not self.position:
          cash = self.broker.getcash()
          position_size = (self.params.margin * self.params.leverage) / self.data.close[0]
          # Long signal
          if self.predicted_value[0] > self.params.buy_threshold:
            self.log('LONG POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.buy(size=position_size)

          # Short signal
          elif self.predicted_value[0] < self.params.sell_threshold:
            self.log('SHORT POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.sell(size=position_size)

        else:
          if self.predicted_value[0] > self.params.buy_threshold and self.position.size < 0:
            self.log('CLOSE SHORT POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.close()
          elif self.predicted_value[0] < self.params.sell_threshold and self.position.size > 0:
            self.log('CLOSE LONG POSITION CREATED, %.2f' % self.datas[0].close[0])
            self.order = self.close()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log('Order Canceled')
            elif order.status == order.Margin:
                self.log(f'Order Margin Not Enough - Available cash: {self.broker.getcash()}')
            elif order.status == order.Rejected:
                self.log('Order Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f'TRADE COMPLETED, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}, Available Cash {self.broker.getcash():.2f}')

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        time = self.datas[0].datetime.time()
        print(f'{dt.isoformat()} {time.isoformat()}, {txt}')


In [74]:
trade_leverage = 25
trade_margin = 1000
trade_buy_threshold = 0.4
trade_sell_threshold = -0.4
broker_commision = 0.02 # In percentage

In [75]:
# Extend PandasData to include the custom column
class CustomPandasData(bt.feeds.PandasData):
    # Add custom columns
    lines = ('predicted_value',)  # Add the custom line
    params = (
        ('predicted_value', 'Predicted_Value'),  # Map the column name
    )


# Load your data into a Pandas DataFrame
backtest_data = CustomPandasData(
    dataname=test_data,
    datetime=None,  # Use the index as the datetime
    open='Open',         # Column index for Open
    high='High',         # Column index for High
    low='Low',          # Column index for Low
    close='Close',        # Column index for Close
    volume='Volume',       # Column index for Volume
    openinterest=None,# No open interest column
    predicted_value='Predicted_Value'  # Column index for Predicted_Value
)

# Create a Cerebro engine instance
cerebro = bt.Cerebro()

# Add the strategy
cerebro.addstrategy(
    PredictedValueStrategy,
    buy_threshold=trade_buy_threshold,
    sell_threshold=trade_sell_threshold,
    leverage=trade_leverage,
    margin=trade_margin
  )

# Add the data feed
cerebro.adddata(backtest_data)

# Set the initial cash
cerebro.broker.set_cash(10000.0)

# Set the commission
cerebro.broker.setcommission(commission=broker_commision / 100)

# Add analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Run the backtest
# Run the backtest
starting_portfolio_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % starting_portfolio_value)
backtest_result = cerebro.run()
final_portfolio_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % final_portfolio_value)

Streaming output truncated to the last 5000 lines.
2024-12-31 15:10:00, BUY EXECUTED, 94996.96
2024-12-31 15:10:00, TRADE COMPLETED, GROSS -114.54, NET -124.57, Available Cash 15705.87
2024-12-31 15:10:00, LONG POSITION CREATED, 95424.92
2024-12-31 15:15:00, Order Margin Not Enough - Available cash: 15705.86911636476
2024-12-31 15:15:00, LONG POSITION CREATED, 95370.95
2024-12-31 15:20:00, Order Margin Not Enough - Available cash: 15705.86911636476
2024-12-31 15:20:00, LONG POSITION CREATED, 95436.99
2024-12-31 15:25:00, Order Margin Not Enough - Available cash: 15705.86911636476
2024-12-31 16:00:00, LONG POSITION CREATED, 95272.29
2024-12-31 16:05:00, Order Margin Not Enough - Available cash: 15705.86911636476
2024-12-31 16:25:00, LONG POSITION CREATED, 94957.28
2024-12-31 16:30:00, Order Margin Not Enough - Available cash: 15705.86911636476
2024-12-31 16:30:00, LONG POSITION CREATED, 94671.44
2024-12-31 16:35:00, Order Margin Not Enough - Available cash: 15705.86911636476
2024-12-31 

In [76]:
strat = backtest_result[0]

# Print analyzers
print(f"Sharpe Ratio: {strat.analyzers.sharpe.get_analysis()}")
print(f"Max Drawdown: {strat.analyzers.drawdown.get_analysis()}")

Sharpe Ratio: OrderedDict([('sharperatio', 1.5869442386992356)])
Max Drawdown: AutoOrderedDict([('len', 146), ('drawdown', 0.5530551315415316), ('moneydown', 304.02891584916506), ('max', AutoOrderedDict([('len', 1706), ('drawdown', 8.44490536590444), ('moneydown', 2536.0838143236324)]))])


In [77]:
cerebro.plot(start=len(test_data)-1000)

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 4 Axes>]]

In [78]:
def make_dict_even(data):
    """
    Recursively makes all numeric values in a dictionary even.

    Args:
        data (dict): The input dictionary (can have nested dictionaries or lists).

    Returns:
        dict: A new dictionary with all numeric values made even.
    """
    if isinstance(data, dict):
        new_dict = {}
        for key, value in data.items():
            new_dict[key] = make_dict_even(value)
        return new_dict
    elif isinstance(data, list):
        return [make_dict_even(item) for item in data]
    elif isinstance(data, (int, float)):
        if isinstance(data, int):
            return data if data % 2 == 0 else data + 1
        else: # float. We will round to an int, and then make even.
            int_value = round(data)
            return int_value if int_value % 2 == 0 else int_value + 1

    else:
        return data  # Return non-numeric values as they are

In [79]:
def fix_out_of_range_values(obj):
    """
    Recursively traverse a nested object and convert out-of-range values to zero.

    Args:
        obj: The input object (can be a dictionary, list, or other nested structure).

    Returns:
        The same object with out-of-range values replaced by zero.
    """
    if isinstance(obj, dict):
        # If the object is a dictionary, recursively process its values
        for key, value in obj.items():
            obj[key] = fix_out_of_range_values(value)
    elif isinstance(obj, list):
        # If the object is a list, recursively process its elements
        for i in range(len(obj)):
            obj[i] = fix_out_of_range_values(obj[i])
    else:
        # If the object is a value, check if it's out of range
        try:
            # Attempt to perform a numeric operation to check if the value is valid
            # For example, check if the value can be compared to a range
            if not (-1e15 < obj < 1e14):  # Adjust the range as needed
                obj = 0
        except (TypeError, ValueError):
            # If the value is not numeric or causes an error, replace it with zero
            obj = 0
    return obj

In [80]:
trade_analysis = make_dict_even(strat.analyzers.tradeanalyzer.get_analysis())
# Errors of large numbers, fix
trade_analysis = fix_out_of_range_values(trade_analysis)
# print
print(json.dumps(trade_analysis, indent=4))

{
    "total": {
        "total": 656,
        "open": 2,
        "closed": 654
    },
    "streak": {
        "won": {
            "current": 0,
            "longest": 20
        },
        "lost": {
            "current": 2,
            "longest": 4
        }
    },
    "pnl": {
        "gross": {
            "total": 51322,
            "average": 78
        },
        "net": {
            "total": 44786,
            "average": 68
        }
    },
    "won": {
        "total": 504,
        "pnl": {
            "total": 75198,
            "average": 150,
            "max": 710
        }
    },
    "lost": {
        "total": 152,
        "pnl": {
            "total": -30412,
            "average": -200,
            "max": -1396
        }
    },
    "long": {
        "total": 242,
        "pnl": {
            "total": 13358,
            "average": 56,
            "won": {
                "total": 26648,
                "average": 144,
                "max": 710
            },
          

In [81]:
drawdown_analysis = make_dict_even(strat.analyzers.drawdown.get_analysis())
print(json.dumps(drawdown_analysis, indent=4))

{
    "len": 146,
    "drawdown": 2,
    "moneydown": 304,
    "max": {
        "len": 1706,
        "drawdown": 8,
        "moneydown": 2536
    }
}


In [82]:
sharpe_analysis = make_dict_even(strat.analyzers.sharpe.get_analysis())
print(json.dumps(sharpe_analysis, indent=4))

{
    "sharperatio": 2
}


In [83]:
def plot_pyfolio_results(results):
    """
    Extracts PyFolio analyzer results from Backtrader and generates interactive plots using Plotly.

    Parameters:
        results (list): The results object returned by Backtrader's cerebro.run().

    Plots:
        1. Cumulative Returns
        2. Drawdowns
        3. Daily Returns
        4. Rolling Sharpe Ratio (252-day)
    """
    # Extract the PyFolio analyzer results
    pyfoliozer = results[0].analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

    # Ensure returns is a pandas Series with a datetime index
    if not isinstance(returns, pd.Series):
        returns = pd.Series(returns, index=pd.date_range(start='2020-01-01', periods=len(returns)))

    # Plot cumulative returns
    cumulative_returns = (1 + returns).cumprod()
    fig = px.line(cumulative_returns, title='Cumulative Returns', labels={'value': 'Cumulative Returns', 'index': 'Date'})
    fig.show()

    # Plot drawdowns
    drawdown = returns / (1 + returns).cumprod() - 1
    fig = px.area(drawdown, title='Drawdowns', labels={'value': 'Drawdown', 'index': 'Date'})
    fig.show()

    # Plot daily returns
    fig = px.bar(returns, title='Daily Returns', labels={'value': 'Daily Returns', 'index': 'Date'})
    fig.show()

In [84]:
plot_pyfolio_results(backtest_result)

In [85]:
def get_model_cloud_storage_path(bucket_name, local_file_name):
    """
    Returns the cloud storage path for a given model name.

    Parameters:
        local_file_name (str): The name of the model.

    Returns:
        str: The cloud storage path. (e.g., gs://<bucket_name>/trade/models/<model_name>.h5).
    """
    return f'gs://{bucket_name}/trade/models/{local_file_name}'

def save_model_to_cloud_storage(model: tf.keras.Model, model_name: str, bucket_name: str):
    """
    Saves a TensorFlow model to Google Cloud Storage and returns the cloud storage file path.

    Parameters:
        model (tf.keras.Model): The TensorFlow model to save.
        model_name (str): The name of the model (used to create the file name).
        bucket_name (str): The name of the Google Cloud Storage bucket.

    Returns:
        str: The cloud storage file path (e.g., gs://<bucket_name>/trade/models/<model_name>.h5).
    """
    # Define the local and cloud storage file paths
    local_file_name = f'{model_name}.h5'
    local_file_path = f'/tmp/{local_file_name}'
    cloud_file_path = get_model_cloud_storage_path(bucket_name, local_file_name)

    # Save the model locally
    model.save(local_file_path)

    try:
        # Upload the model to Google Cloud Storage
        subprocess.run(['gsutil', 'cp', local_file_path, cloud_file_path], check=True)
        print(f"Model saved to {cloud_file_path}")
    except subprocess.CalledProcessError as e:
        # Handle errors during the upload process
        raise RuntimeError(f"Failed to upload model to Google Cloud Storage: {e}")
    finally:
        # Clean up the local file
        if os.path.exists(local_file_path):
            os.remove(local_file_path)
    return cloud_file_path

def load_model_from_cloud_storage(model_name: str, bucket_name: str):
    """
    Loads a TensorFlow model from Google Cloud Storage.

    Parameters:
        model_name (str): The name of the model (used to create the file name).
        bucket_name (str): The name of the Google Cloud Storage bucket.

    Returns:
        tf.keras.Model: The loaded TensorFlow model.
    """
    # Define the local and cloud storage file paths
    local_file_name = f'{model_name}.h5'
    local_file_path = f'/tmp/{local_file_name}'
    cloud_file_path = get_model_cloud_storage_path(bucket_name, local_file_name)

    try:
        # Download the model from Google Cloud Storage
        subprocess.run(['gsutil', 'cp', cloud_file_path, local_file_path], check=True)

        # Load the model
        model = tf.keras.models.load_model(local_file_path)
        print(f"Model loaded from {cloud_file_path}")
        return model
    except subprocess.CalledProcessError as e:
        raise FileNotFoundError(f"Model not found in Google Cloud Storage: {e}")
    finally:
        # Clean up the local file
        if os.path.exists(local_file_path):
            os.remove(local_file_path)


In [ ]:
todays_date = datetime.datetime.now().strftime("%Y-%m-%d")
random_hash = generate_random_hash()
model_name = f"binance_mtf_prod_{timeframe}_{start_date}_{end_date}_{random_hash}"
model_path = save_model_to_cloud_storage(model, model_name, bucket_name)

In [ ]:
analysis_result = {
    'timeframe': timeframe,
    'start_date': start_date,
    'end_date': end_date,
    'symbol': symbol,
    'exchange': exchange,
    'features_columns': features_columns,
    'lower_timeframe': lower_timeframe,
    'middle_timeframe': middle_timeframe,
    'higher_timeframe': higher_timeframe,
    'seq_length': seq_length,
    'training_data_ratio': training_data_ratio,
    'epochs': train_epochs,
    'model_input_shape': model_input_shape,
    'model_name': model_name,
    'model_path': model_path,
    'train_loss': train_loss,
    'train_mae': train_mae,
    'val_loss': val_loss,
    'val_mae': val_mae,
    'final_epoch': final_epoch,
    'training_data_start_date': training_data_start_date,
    'training_data_end_date': training_data_end_date,
    'test_data_start_date': test_data_start_date,
    'test_data_end_date': test_data_end_date,
    'trade_leverage': trade_leverage,
    'trade_margin': trade_margin,
    'trade_buy_threshold': trade_buy_threshold,
    'trade_sell_threshold': trade_sell_threshold,
    'starting_portfolio_value': starting_portfolio_value,
    'final_portfolio_value': final_portfolio_value,
    'broker_commision': broker_commision,
    'trade_analysis': trade_analysis,
    'drawdown_analysis': drawdown_analysis,
    'sharpe_analysis': sharpe_analysis,
}
analysis_result

In [ ]:
def initialize_firestore(project_id):
    """
    Initialize the Firestore client using the service account key.

    Parameters:
        project_id (str): The Google Cloud project ID.

    Returns:
        firestore.Client: Initialized Firestore client.
    """
    print(f"project id {project_id}")
    if not firebase_admin._apps:
        # cred = credentials.Certificate(service_account_key_path)
        cred = firebase_admin.credentials.ApplicationDefault()
        cred._project_id = project_id  # Add this line
        os.environ["GOOGLE_CLOUD_PROJECT"] = project_id
        firebase_admin.initialize_app(cred, {'projectId': project_id})
    return firestore.client()


def save_to_firestore(project_id, collection_name, data):
    """
    Save data to a Firestore collection.

    Parameters:
        project_id (str): The Google Cloud project ID.
        service_account_key_path (str): Path to the Firebase service account key JSON file.
        collection_name (str): Name of the Firestore collection.
        data (dict): Data to save in the document.

    Returns:
        None
    """
    # Initialize Firestore client
    db = initialize_firestore(project_id)

    # Save data to Firestore
    doc_ref = db.collection(collection_name).document()
    doc_ref.set(data)
    print(f"Data saved to Firestore: Collection={collection_name}, Document ID={doc_ref.id}")

In [ ]:
save_to_firestore(project_id, firestore_collection_name, analysis_result)